# Import des modules

In [1]:
import glob
import pandas as pd

# Lecture des données 

Données venant du Sirsé et cartosanté

In [2]:
liste = glob.glob('*.csv')
for i in range(len(liste)):
    if i == 0 :
        db = pd.read_csv(liste[i], skiprows=2, sep=';', encoding="utf-8", low_memory=False)
    else:
        db_temp = pd.read_csv(liste[i], skiprows=2, sep=';', encoding="utf-8", low_memory=False)
        try:
            db_temp.drop('Libellé',axis=1, inplace=True)
        except:
            pass
        db = db.merge(db_temp, left_on='Code', right_on='Code', how='inner')

Données de l'insee sur les codes de départements et regions

In [3]:
dep = pd.read_csv("../dep/communes-departement-region.csv", sep=",", encoding="utf-8")
#dep = dep.groupby(['code_commune_INSEE']).count().reset_index()
dep['code_commune_INSEE'] = dep['code_commune_INSEE'].apply(lambda x: '0'+str(x) if(len(str(x)) == 4) else str(x))
dep['code_postal'] = dep['code_postal'].apply(lambda x: '0'+str(x) if(len(str(x)) == 4) else str(x))
dep['code_dep'] = dep['code_commune_INSEE'].str[:2]
dep = dep[['code_dep','code_commune_INSEE', 
     'nom_departement','nom_region','code_postal',
     'code_region','longitude','latitude']]

# Modification des noms des colonnes

In [4]:
db.columns

Index(['Code', 'Libellé', 'Nbre de généralistes lib. au 31/12 2019',
       'Nbre de généralistes lib. au 31/12 2019.1',
       'Population des moins de 18 ans 2016',
       'Nombre de chômeurs chez les 15-64 ans (RP) 2016',
       'Nombre de chômeurs chez les 15-24 ans (RP) 2016',
       'Nombre de résidences principales en HLM 2016',
       'Revenu fiscal médian 2016', 'Population des 65 ans ou plus 2016',
       'Taux de natalité 2016', 'Population municipale 2016'],
      dtype='object')

In [5]:
db.columns = ['Code','Nom_com', 'NG19_60>', 'NG19T',
              'Pop16_<18', 'NC16_15_64','NC16_15_24',
              'NRP16','RFM16','Pop16_>65','TN16','PM16']

# Jointure des données

In [6]:
merged_data = pd.merge(db,dep, left_on= "Code", right_on ="code_commune_INSEE", how="left")
data = pd.DataFrame(merged_data['Code'].value_counts() == 1)
liste_code_commune = data[data['Code']==True].index
merged_data_one = merged_data[merged_data['Code'].isin(liste_code_commune)]
#merged_data_one['code_region'] = merged_data_one['code_region'].astype(int).astype(str)
merged_data_one.drop(["Code","longitude","latitude"], axis=1, inplace=True)

C:\Users\joaka\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
merged_data_one.to_csv("output_data.csv")